# Active learning loop

# Imports

In [1]:
import numpy as np
import os

# torch
import torch

# HF
from datasets import load_dataset, Features, Value, ClassLabel, Version
from transformers import AutoTokenizer

# small text
from small_text import random_initialization
from small_text.active_learner import PoolBasedActiveLearner
from small_text.base import LABEL_UNLABELED
from small_text.integrations.transformers import TransformersDataset, TransformerModelArguments
from small_text.integrations.transformers.classifiers.factories import TransformerBasedClassificationFactory
from small_text.query_strategies import BreakingTies

# rubrix
import rubrix as rb
from rubrix.listeners import listener

# metrics
from sklearn.metrics import accuracy_score

/src/.venv/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Constants

In [2]:
DATASET_AG_NEWS = "bergr7/weakly_supervised_ag_news"
TRANSFORMER_MODEL = "distilbert-base-uncased"
TOKENIZER = AutoTokenizer.from_pretrained(TRANSFORMER_MODEL)
LABELS = load_dataset('ag_news')["train"].features["label"].names
NUM_SAMPLES_ITER = 5
NUM_CLASSES = len(LABELS)
DATASET_NAME = "active_learning_loop"

2022-10-13 14:46:10.376 | WARNING  | datasets.builder:_create_builder_config:473 - Using custom data configuration default
2022-10-13 14:46:10.400 | WARNING  | datasets.builder:download_and_prepare:732 - Found cached dataset ag_news (/root/.cache/huggingface/datasets/ag_news/default/0.0.0/bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548)
100%|██████████| 2/2 [00:00<00:00, 713.56it/s]


# Load data

In [3]:
# labeled data - train.csv contains weak labels
labeled_data_files = {
    "train": "train.csv",
    "validation": "validation.csv", 
    "test": "test.csv"
}

# unlabeled data - not covered by LFs
unlabeled_data_files = {"unlabeled": "unlabeled_train.csv"}

# Define schema
labeled_features = Features(
    {
        "text": Value("string"),
        "label": ClassLabel(
            num_classes=4,
            names=['World', 'Sports', 'Business', 'Sci/Tech']
        )
    }
)
unlabeled_features = Features({"text": Value("string")})

# load data
labeled_dataset = load_dataset(
    DATASET_AG_NEWS,
    data_files=labeled_data_files,
    features=labeled_features
)

unlabeled_dataset = load_dataset(
    DATASET_AG_NEWS,
    data_files=unlabeled_data_files,
    features=unlabeled_features
)

2022-10-13 14:46:14.687 | WARNING  | datasets.builder:_create_builder_config:473 - Using custom data configuration bergr7--weakly_supervised_ag_news-6f78f309523478bd
2022-10-13 14:46:14.691 | WARNING  | datasets.builder:download_and_prepare:732 - Found cached dataset csv (/root/.cache/huggingface/datasets/bergr7___csv/bergr7--weakly_supervised_ag_news-6f78f309523478bd/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a)
100%|██████████| 3/3 [00:00<00:00, 871.69it/s]
2022-10-13 14:46:14.823 | WARNING  | datasets.builder:_create_builder_config:473 - Using custom data configuration bergr7--weakly_supervised_ag_news-9442e7dc9bdd01c3
2022-10-13 14:46:14.824 | WARNING  | datasets.builder:download_and_prepare:732 - Found cached dataset csv (/root/.cache/huggingface/datasets/bergr7___csv/bergr7--weakly_supervised_ag_news-9442e7dc9bdd01c3/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a)
100%|██████████| 1/1 [00:00<00:00, 835.19it/s]


In [4]:
labeled_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 37340
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 24000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 7600
    })
})

In [5]:
unlabeled_dataset

DatasetDict({
    unlabeled: Dataset({
        features: ['text'],
        num_rows: 58660
    })
})

# Training data


In [34]:
# text
weak_train_text = [row["text"] for row in labeled_dataset["train"]]
unlabeled_train_text = [row["text"] for row in unlabeled_dataset["unlabeled"]]
# labels
weak_train_labels = [row["label"] for row in labeled_dataset["train"]]
unlabeled_no_labels = [LABEL_UNLABELED for _ in unlabeled_train_text]
# dataset
train_text = np.array(weak_train_text + unlabeled_train_text)
train_labels = np.array(weak_train_labels + unlabeled_no_labels)

# Preprocessing the dataset

Wrap the dataset in a specific data class by small-text -> `TransformersDataset` since we are gonna use `distilbert-base-uncased`

`TransformersDataset` should contain the tokenized input text.


In [35]:
# create dataset
dataset = TransformersDataset.from_arrays(
    train_text,
    train_labels,
    tokenizer=TOKENIZER,
    target_labels=[0, 1, 2, 3]
)

/src/.venv/lib/python3.10/site-packages/small_text/utils/annotations.py:67: ExperimentalWarning: The function from_arrays is experimental and maybe subject to change soon.
  warnings.warn(f'The {subject} {func_or_class.__name__} is experimental '


In [36]:
# visualize an input - weak label
print(f"input_ids: {dataset.data[37339][0]}")
print(f"token_type_ids: {dataset.data[37339][1]}")
print(f"label: {dataset.data[37339][2]}")

# visualize an input - unlabeled
print(f"input_ids: {dataset.data[50000][0]}")
print(f"token_type_ids: {dataset.data[50000][1]}")
print(f"label: {dataset.data[50000][2]}")

input_ids: tensor([[  101, 19817, 15441,  2099, 13066,  6518, 12487,  2377,  2859,  1001,
          4464,  1025,  1055, 19817, 15441,  2099,  9167,  2545,  2024, 25919,
          3209,  2114,  1996,  5081,  1997,  1037,  2149,  3119,  2162,  6461,
          2012,  2037,  3688,  1012,  2149,  4584,  2441,  2019,  4812,  2006,
          5958,  2046,  3251,  2822, 19817, 15441,  2099, 11153,  2020,  4979,
          2075,  2019, 15571,  3745,  1997,  1996,  2088,  3006,  1012,   102,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     

In [8]:
# validation dataset
val_text = [row["text"] for row in labeled_dataset["validation"]][:20]
val_labels = [row["label"] for row in labeled_dataset["validation"]][:20]

In [9]:
val_dataset = TransformersDataset.from_arrays(
    val_text,
    val_labels,
    tokenizer=TOKENIZER,
    target_labels=[0, 1, 2, 3]
)

>Notes

- cleaning??
- test set???

# Set up active learner

- Component 1 -> classifier
- Component 2 -> query strategy (sampling)

In [18]:
# hyper-params
params = dict(
        {
            "lr": 3e-5,
            "num_epochs": 1,
            "mini_batch_size": 32,
            "model_selection": True,
            "device": "cuda"
    }
)

# classifier
model_factory = TransformerBasedClassificationFactory(
    TransformerModelArguments(TRANSFORMER_MODEL),
    num_classes=NUM_CLASSES,
    kwargs=params
)

In [19]:
# define query strategy
query_strategy = BreakingTies()

In [20]:
def initialize_active_learner(active_learner, train_labels, weak_train_labels):

    indices_initial = [i for i in range(len(weak_train_labels))]
    active_learner.initialize_data(indices_initial, train_labels[indices_initial])

    return indices_initial

In [21]:
active_learner = PoolBasedActiveLearner(
    model_factory,
    query_strategy,
    dataset,
    reuse_model=True
)

In [23]:
# this takes quite a bit of time... (1hr!!) - we initilize the active learner with the weak labels
indices_labeled = initialize_active_learner(active_learner, train_labels, weak_train_labels)

Since most query strategies, including ours, require a trained model, we randomly draw a subset from the data pool to initialize our AL system. After obtaining the labels for this batch of instances, the active learner will use them to create the first classifier.

# Active Learning loop with Rubrix

Configure a Rubrix dataset and set up active learning loop

In [15]:
# init rubrix

# RUBRIX_URL = os.getenv("RUBRIX_API_URL", "http://localhost:6900")

rb.init(api_url="http://rubrix:80")

In [16]:
settings = rb.TextClassificationSettings(label_schema=LABELS)
# configure dataset
rb.configure_dataset(name=DATASET_NAME, settings=settings)

In [27]:
# first batch querying with active learner
batch_1_indices = active_learner.query(num_samples=NUM_SAMPLES_ITER)

records = [
    rb.TextClassificationRecord(
        text=train_text[idx],
        metadata={"batch_id": 0},
        id=idx,
    )
    for idx in batch_1_indices
]

# Log initial records to Rubrix
rb.log(records, DATASET_NAME)

100%|██████████| 5/5 [00:00<00:00, 27.75it/s]

5 records logged to http://rubrix:80/datasets/rubrix/active_learning_loop


BulkResponse(dataset='active_learning_loop', processed=5, failed=0)

In [28]:
LABEL2INT = labeled_dataset["train"].features["label"].str2int

In [29]:
ACCURACIES = []

In [30]:
@listener(dataset=DATASET_NAME,
    query="status:Validated AND metadata.batch_id:{batch_id}",
    condition=lambda search: search.total==NUM_SAMPLES_ITER,
    execution_interval_in_seconds=3,
    batch_id=0
)
def active_learning_loop(records, ctx):
    
    # 1. Update active learner
    print(f"Updating with batch_id {ctx.query_params['batch_id']} ...")
    y = np.array([LABEL2INT(rec.annotation) for rec in records])

    # update with the prior queried indices
    active_learner.update(y)
    print("Done!")

    # 2. Query active learner
    print("Querying new data points ...")
    queried_indices = active_learner.query(num_samples=NUM_SAMPLES_ITER)
    new_batch = ctx.query_params["batch_id"] + 1
    new_records = [
        rb.TextClassificationRecord(
            text=train_text[idx],
            metadata={"batch_id": new_batch},
            id=idx,
        )
        for idx in queried_indices
    ]

    # 3. Log the batch to Rubrix
    rb.log(new_records, DATASET_NAME)

    # 4. Evaluate current classifier on the test set
    print("Evaluating current classifier ...")
    accuracy = accuracy_score(
        val_dataset.y,
        active_learner.classifier.predict(val_dataset),
    )

    ACCURACIES.append(accuracy)
    ctx.query_params["batch_id"] = new_batch
    print("Done!")

    print("Waiting for annotations ...")

# Start active learning loop

In [31]:
active_learning_loop.start()

Updating with batch_id 0 ...
Done!
Querying new data points ...


100%|██████████| 5/5 [00:00<00:00, 13.61it/s]


5 records logged to http://rubrix:80/datasets/rubrix/active_learning_loop
Evaluating current classifier ...
Done!
Waiting for annotations ...


In [32]:
active_learning_loop.stop()